# Задание 1

In [1]:
import pandas as pd
import sqlalchemy
import numpy as np
# Подключаемся с серверу SQL
engine_connection_string = ''
engine = sqlalchemy.create_engine(engine_connection_string, fast_executemany=True)

In [2]:
df = pd.read_csv(r'data/shedulers.csv', encoding='cp1251', sep=';')
df.head(10)

,ФИО,Расписание,Дата начала расписания,Дата окончания расписания
0,Поставщик 1,дддвсвнн,01.01.2019 0:00,10.01.2019 0:00
1,Поставщик 1,ннвннв,11.01.2019 0:00,15.01.2019 0:00
2,Поставщик 1,св,16.01.2019 0:00,20.01.2019 0:00
3,Поставщик 2,свсвсв,01.01.2019 0:00,07.01.2019 0:00
4,Поставщик 2,днвсв,08.01.2019 0:00,14.01.2019 0:00
5,Поставщик 2,ннддвсв,15.01.2019 0:00,31.12.9999 0:00
6,Поставщик 3,нвнвнв,01.01.2019 0:00,01.02.2019 0:00
7,Поставщик 3,двдвдвдв,02.02.2019 0:00,31.12.9999 0:00


In [3]:
# Переименовываем колонки и заливаем базовую таблицу на сервер
df.columns = ['NAME', 'SHEDULE', 'DATE_BEGIN', 'DATE_END']
df.to_sql('AVILOV_BASETABLE', 
                engine, 
                index = False,
                schema = 'dbo', 
                if_exists = 'replace')

In [4]:
query = \
'''
SELECT DISTINCT(SUBSTRING(T.NAME, 11, LEN(NAME)-10)) AS ID_NAME,  -- Выбираем конец строки (то есть число) как id поставщика
        T.NAME, -- выбираем имя поставщика
        STRING_AGG(SHEDULE, '') AS SHEDULE, -- собираем все известные расписания в одно
        MIN(DATE_BEGIN) AS DATE_BEGIN, -- начало расписание как минимальная дата начала расписания
        MAX(DATE_END) AS DATE_END -- конец расписание как максимальнаяя дата начала расписания
INTO [T_CONTRACTOR_SHEDULER] -- заливаем отобранную таблицу в таблицу T_CONTRACTOR_SHEDULER
FROM [AVILOV_BASETABLE] AS T
GROUP BY T.NAME
'''

# Исполням запрос
with engine.connect() as connection:
        tr = connection.begin()
        result = connection.execute(query)
        tr.commit()

# Задание 2

In [5]:
query = \
'''
-- Создаем пустую таблицу для заполнения рабочими днями
CREATE TABLE [T_CONTRACTOR_WORK_DAY]
(
ID INT,
[NAME] varchar(MAX),
DATE_BEGIN DATETIME,
DATE_END DATETIME
)

DECLARE @current_id INT = 1; -- Текущий id поставщика
DECLARE @current_row INT = 1; -- Текущий id строки (для заполнения в финальной таблице)

-- Повторяем для каждого поставщика
WHILE @current_id in (SELECT ID_NAME FROM [T_CONTRACTOR_SHEDULER])
BEGIN

    DECLARE @i INT = 1; -- Счетчик рабочих дней (включая выходные)
    DECLARE @count int; -- Количество рабочих дней (включая выходные) в таблице T_CONTRACTOR_SHEDULER
    DECLARE @current_shift_start DATETIME; -- Начало текущего рабочего дня
    DECLARE @current_shift_end DATETIME; -- Конец текущего рабочего дня
    DECLARE @current_shedule varchar; -- Текущая смена (дневная/ночная/суточная/выходной)
    DECLARE @name varchar(MAX); -- имя текущего поставщика
    SELECT @count =  LEN(SHEDULE) FROM [T_CONTRACTOR_SHEDULER] WHERE ID_NAME = @current_id;
    SELECT @current_shift_start =  DATE_BEGIN FROM [T_CONTRACTOR_SHEDULER] WHERE ID_NAME = @current_id;
    SELECT @current_shift_end =  @current_shift_start;
    SELECT @name = [NAME] FROM [T_CONTRACTOR_SHEDULER] WHERE ID_NAME = @current_id;
    
    -- Выполняем пока не пройдемся по всему расписанию
    WHILE @i <= @count
    BEGIN
        
        -- Выбираем текущую смену
        SELECT @current_shedule = SUBSTRING(SHEDULE, @i, 1) FROM [T_CONTRACTOR_SHEDULER] WHERE ID_NAME = @current_id
        
        IF @current_shedule = 'д'
        BEGIN
            -- Если предыдущая смена (именно на нее указывает current_shift_start) закончилась позже 8 утра, то округляем
            -- то округляем эту дату до следующей полуночи
            IF DATEPART(hour, @current_shift_start) > 8
                SET @current_shift_start = DATEADD(Day, DATEDIFF(Day, 0, @current_shift_start), 1)
            -- Если предыдущая смена закончилась раньше 8 утра, то округляем до предыдущей полуночи
            ELSE
                SET @current_shift_start = DATEADD(Day, DATEDIFF(Day, 0, @current_shift_start), 0)
            -- Добавляем 8 часов к началу рабочей смены (смена начинается в 8 утра, current_shift_start гарантировано на полуночи)
            SET @current_shift_start = DATEADD(HOUR, 8, @current_shift_start)
            -- Добавляем 12 часов к началу смены и получаем окончание смены (именно столько длится смена)
            SET @current_shift_end = DATEADD(HOUR, 12, @current_shift_start)
        END
        
        IF @current_shedule = 'н'
        BEGIN
            -- Если предыдущая смена позже 20:00 то округляем то округляем эту дату до следующей полуночи
            IF DATEPART(hour, @current_shift_start) > 20
                SET @current_shift_start = DATEADD(Day, DATEDIFF(Day, 0, @current_shift_start), 1)
            -- Если предыдущая смена закончилась раньше 20:00, то округляем до предыдущей полуночи
            ELSE
                SET @current_shift_start = DATEADD(Day, DATEDIFF(Day, 0, @current_shift_start), 0)
            -- Добавляем 20 часов к началу рабочей смены (смена начинается в 20:00, current_shift_start гарантировано на полуночи)
            SET @current_shift_start = DATEADD(HOUR, 20, @current_shift_start)
            -- Добавляем 12 часов к началу смены и получаем окончание смены (именно столько длится смена)
            SET @current_shift_end = DATEADD(HOUR, 12, @current_shift_start)
        END
        
        IF @current_shedule = 'с'
        BEGIN
            -- Если предыдущая смена позже 8 утра то округляем то округляем эту дату до следующей полуночи
            IF DATEPART(hour, @current_shift_start) > 8
                SET @current_shift_start = DATEADD(Day, DATEDIFF(Day, 0, @current_shift_start), 1)
            -- Если предыдущая смена закончилась раньше 8 утра, то округляем до предыдущей полуночи
            ELSE
                SET @current_shift_start = DATEADD(Day, DATEDIFF(Day, 0, @current_shift_start), 0)
            -- Добавляем 8 часов к началу рабочей смены (смена начинается в 8 утра, current_shift_start гарантировано на полуночи)
            SET @current_shift_start = DATEADD(HOUR, 8, @current_shift_start)
            -- Добавляем 24 часов к началу смены и получаем окончание смены (именно столько длится смена)
            SET @current_shift_end = DATEADD(HOUR, 24, @current_shift_start)
        END
        
        IF @current_shedule = 'в'
        -- Если текущая смена - выходной, то следующие 24 часа рабочей смены не будет
        BEGIN
            SET @current_shift_end = DATEADD(hour, 24, @current_shift_start)
        END
        
        IF @current_shedule != 'в'
        -- Если текущая смена не выходной, то добавляем в таблицу запись о смене
        BEGIN
            INSERT INTO [T_CONTRACTOR_WORK_DAY]
                VALUES (@current_row, @name, @current_shift_start, @current_shift_end)
            --Обновляем id строки в таблице
            SET @current_row = @current_row + 1;
        END
        
        -- Обновляем счетчик текущей смены
        SET @i = @i + 1
        --
        SET @current_shift_start = @current_shift_end
    END
    
    SET @current_id = @current_id + 1;
    
END
'''

In [6]:
# Исполням запрос
with engine.connect() as connection:
        tr = connection.begin()
        result = connection.execute(query)
        tr.commit()

# Задание 3

In [7]:
query = \
'''
-- Дропаем созданную на предыдущем шаге таблицу
DROP TABLE [T_CONTRACTOR_WORK_DAY]

-- Создаем пустую таблицу для заполнения рабочими днями
CREATE TABLE [T_CONTRACTOR_WORK_DAY]
(
ID INT,
[NAME] varchar(MAX),
DATE_BEGIN DATETIME,
DATE_END DATETIME
)

--Создаем процедуру с параметрами
USE [SANDBOX_DB_TEAM_3];
GO
CREATE PROCEDURE WORKING_DAYS (
                @shedule_start DATETIME,
                @shedule_end DATETIME)
AS
BEGIN

DECLARE @current_id INT = 1; -- Текущий id поставщика
DECLARE @current_row INT = 1; -- Текущий id строки (для заполнения в финальной таблице)

-- Повторяем для каждого поставщика
WHILE @current_id in (SELECT ID_NAME FROM [T_CONTRACTOR_SHEDULER])
BEGIN

    DECLARE @i INT = 1; -- Счетчик рабочих дней (включая выходные)
    DECLARE @count int; -- Количество рабочих дней (включая выходные) в таблице T_CONTRACTOR_SHEDULER
    DECLARE @current_shift_start DATETIME; -- Начало текущего рабочего дня
    DECLARE @current_shift_end DATETIME; -- Конец текущего рабочего дня
    DECLARE @current_shedule varchar; -- Текущая смена (дневная/ночная/суточная/выходной)
    DECLARE @name varchar(MAX); -- имя текущего поставщика
    SELECT @count =  LEN(SHEDULE) FROM [T_CONTRACTOR_SHEDULER] WHERE ID_NAME = @current_id;
    SELECT @current_shift_start =  @shedule_start;
    SELECT @current_shift_end =  @current_shift_start;
    SELECT @name = [NAME] FROM [T_CONTRACTOR_SHEDULER] WHERE ID_NAME = @current_id;
    
    -- Выполняем пока не пройдемся по всему заданному периоду
    WHILE @current_shift_end<@shedule_end
    BEGIN
    
        -- Выбираем текущую смену
        SELECT @current_shedule = SUBSTRING(SHEDULE, @i, 1) FROM [T_CONTRACTOR_SHEDULER] WHERE ID_NAME = @current_id
    
        IF @current_shedule = 'д'
        BEGIN
            -- Если предыдущая смена (именно на нее указывает current_shift_start) закончилась позже 8 утра, то округляем
            -- то округляем эту дату до следующей полуночи
            IF DATEPART(hour, @current_shift_start) > 8
                SET @current_shift_start = DATEADD(Day, DATEDIFF(Day, 0, @current_shift_start), 1)
            -- Если предыдущая смена закончилась раньше 8 утра, то округляем до предыдущей полуночи
            ELSE
                SET @current_shift_start = DATEADD(Day, DATEDIFF(Day, 0, @current_shift_start), 0)
            -- Добавляем 8 часов к началу рабочей смены (смена начинается в 8 утра, current_shift_start гарантировано на полуночи)
            SET @current_shift_start = DATEADD(HOUR, 8, @current_shift_start)
            -- Добавляем 12 часов к началу смены и получаем окончание смены (именно столько длится смена)
            SET @current_shift_end = DATEADD(HOUR, 12, @current_shift_start)
        END
        
        IF @current_shedule = 'н'
        BEGIN
            -- Если предыдущая смена позже 20:00 то округляем то округляем эту дату до следующей полуночи
            IF DATEPART(hour, @current_shift_start) > 20
                SET @current_shift_start = DATEADD(Day, DATEDIFF(Day, 0, @current_shift_start), 1)
            -- Если предыдущая смена закончилась раньше 20:00, то округляем до предыдущей полуночи
            ELSE
                SET @current_shift_start = DATEADD(Day, DATEDIFF(Day, 0, @current_shift_start), 0)
            -- Добавляем 20 часов к началу рабочей смены (смена начинается в 20:00, current_shift_start гарантировано на полуночи)
            SET @current_shift_start = DATEADD(HOUR, 20, @current_shift_start)
            -- Добавляем 12 часов к началу смены и получаем окончание смены (именно столько длится смена)
            SET @current_shift_end = DATEADD(HOUR, 12, @current_shift_start)
        END
        
        IF @current_shedule = 'с'
        BEGIN
            -- Если предыдущая смена позже 8 утра то округляем то округляем эту дату до следующей полуночи
            IF DATEPART(hour, @current_shift_start) > 8
                SET @current_shift_start = DATEADD(Day, DATEDIFF(Day, 0, @current_shift_start), 1)
            -- Если предыдущая смена закончилась раньше 8 утра, то округляем до предыдущей полуночи
            ELSE
                SET @current_shift_start = DATEADD(Day, DATEDIFF(Day, 0, @current_shift_start), 0)
            -- Добавляем 8 часов к началу рабочей смены (смена начинается в 8 утра, current_shift_start гарантировано на полуночи)
            SET @current_shift_start = DATEADD(HOUR, 8, @current_shift_start)
            -- Добавляем 24 часов к началу смены и получаем окончание смены (именно столько длится смена)
            SET @current_shift_end = DATEADD(HOUR, 24, @current_shift_start)
        END
        
        IF @current_shedule = 'в'
        -- Если текущая смена - выходной, то следующие 24 часа рабочей смены не будет
        BEGIN
            SET @current_shift_end = DATEADD(hour, 24, @current_shift_start)
        END
        
        IF @current_shedule != 'в'
        -- Если текущая смена не выходной, то добавляем в таблицу запись о смене
        BEGIN
            -- Проверяем, что конец текущей смены не выходит за границу заданного периода
            IF @current_shift_end<@shedule_end
            BEGIN
            INSERT INTO [T_CONTRACTOR_WORK_DAY]
                VALUES (@current_row, @name, @current_shift_start, @current_shift_end)
            --Обновляем id строки в таблице
            SET @current_row = @current_row + 1;
            END
            
        END
        -- Обновляем счетчик текущей смены
        SET @i = @i + 1
        
        -- Если в расписании закончились смены, то возвращаемся в начало расписания
        IF @i>@count
        BEGIN
        SET @i = 1
        END
        
        -- Передвигаем указать начала текущей смены на конец текущей смены
        SET @current_shift_start = @current_shift_end
    END
    -- Обновляем счетчик поставщиков
    SET @current_id = @current_id + 1;
    
END

END
'''

In [ ]:
# Исполням запрос
with engine.connect() as connection:
        tr = connection.begin()
        result = connection.execute(query)
        tr.commit()

In [ ]:
## Заоплним таблицу значениями
query = \
'''
declare @shedule_start DATETIME = '2019-01-01';
declare @shedule_end DATETIME = '2019-03-01';
EXEC WORKING_DAYS @shedule_start, @shedule_end
'''

In [ ]:
# Исполням запрос
with engine.connect() as connection:
        tr = connection.begin()
        result = connection.execute(query)
        tr.commit()

# Задание 4

In [8]:
# Считаем сколько рабочих дней было у каждого из поставщиков
query = \
'''
SELECT [NAME], COUNT(*) AS working_days
FROM [T_CONTRACTOR_WORK_DAY]
GROUP BY([NAME])
'''
pd.read_sql(query, con = engine)

,NAME,working_days
0,Поставщик 1,41
1,Поставщик 2,38
2,Поставщик 3,32


In [9]:
# Находим поставщиков, у которых в янврае 2019 было больше 10 рабочих дней
query = \
'''
SELECT [NAME]
FROM [T_CONTRACTOR_WORK_DAY]
-- Выбираем только наблюдения из января 2019
WHERE DATEPART(YEAR, DATE_BEGIN) = 2019 AND DATEPART(MONTH, DATE_BEGIN) = 1
GROUP BY ([NAME])
-- Выбираем только поставиков, укоторых более 10 рабочих дней
HAVING COUNT(*)>10
'''
pd.read_sql(query, con = engine)

,NAME
0,Поставщик 1
1,Поставщик 2
2,Поставщик 3


In [10]:
# Выбираем поставщиков, которые работыли 14, 15 и 16 января 2019
query = \
'''
SELECT [NAME]
FROM [T_CONTRACTOR_WORK_DAY] AS T
-- Выбираем только поставщиков, у которых были рабочие дни 14, 15 и 16 числа
WHERE ([NAME] IN (SELECT [NAME] FROM [[T_CONTRACTOR_WORK_DAY] WHERE DATE_BEGIN<'2019-01-15' AND DATE_END>='2019-01-14')) AND
([NAME] IN (SELECT [NAME] FROM [T_CONTRACTOR_WORK_DAY] WHERE DATE_BEGIN<='2019-01-16' AND DATE_END>='2019-01-15')) AND
([NAME] IN (SELECT [NAME] FROM [T_CONTRACTOR_WORK_DAY] WHERE DATE_BEGIN<='2019-01-17' AND DATE_END>='2019-01-16'))
GROUP BY [NAME]
'''
pd.read_sql(query, con = engine)

,NAME
0,Поставщик 1
1,Поставщик 3


# Задание 5

In [11]:
# Загружаем таблицу с раписанием в память
t_contractor_work_day = pd.read_sql('SELECT * FROM [T_CONTRACTOR_WORK_DAY]', con = engine)
t_contractor_work_day['DATE_BEGIN'] = t_contractor_work_day['DATE_BEGIN'].dt.date
t_contractor_work_day['DATE_END'] = t_contractor_work_day['DATE_END'].dt.date
t_contractor_work_day.head()

,ID,NAME,DATE_BEGIN,DATE_END
0,1,Поставщик 1,2019-01-01,2019-01-01
1,2,Поставщик 1,2019-01-02,2019-01-02
2,3,Поставщик 1,2019-01-03,2019-01-03
3,4,Поставщик 1,2019-01-05,2019-01-06
4,5,Поставщик 1,2019-01-07,2019-01-08


In [12]:
# Считаем сколько рабочих дней было у каждого из поставщиков
t_contractor_work_day.groupby('NAME', as_index = False).size()

,NAME,size
0,Поставщик 1,41
1,Поставщик 2,38
2,Поставщик 3,32


In [16]:
# Находим поставщиков, у которых в янврае 2019 было больше 10 рабочих дней
working_days_january = t_contractor_work_day[(t_contractor_work_day['DATE_BEGIN']>pd.to_datetime('2019-01-01')) \
                                             & (t_contractor_work_day['DATE_END']<pd.to_datetime('2019-02-01'))]
working_days_january.groupby('NAME', as_index = False).filter(lambda x: x['DATE_BEGIN'].count()>10)['NAME'].unique()

array(['Поставщик 1', 'Поставщик 2', 'Поставщик 3'], dtype=object)

In [17]:
# Выбираем поставщиков, которые работыли 14, 15 и 16 января 2019
def filtering_working_days(x):
    '''
    x (dateframe) - датарфрейм для одного поставщика
    return - False/True зависимости от того, работал ли поставщик 14, 16 и 16 января 2019
    '''
    intersect = np.intersect1d([pd.to_datetime(x) for x in ['2019-01-14', '2019-01-15', '2019-01-16']], 
                               pd.concat([x['DATE_BEGIN'], x['DATE_END']]).values)
    
    return len(intersect)==3

t_contractor_work_day.groupby('NAME', as_index = False).filter(filtering_working_days)['NAME'].unique()

array(['Поставщик 1', 'Поставщик 3'], dtype=object)